In [107]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [108]:
df_met = pd.read_csv('data/clean/mean_met.csv')
df_demand = pd.read_csv('data/clean/curva_horaria.csv')

In [109]:
df_demand = df_demand.drop(columns='Unnamed: 0')

In [110]:
time_strs = df_met['time'].astype(str).str.zfill(4)
df_met['date'] = pd.to_datetime(df_met['date'])
df_met['time'] = pd.to_datetime(time_strs, format="%H%M")
df_met['time'] = df_met['time'].dt.time
df_demand['date'] = pd.to_datetime(df_demand['date'])
df_demand['time'] = pd.to_datetime(time_strs, format="%H%M")
df_demand['time'] = df_demand['time'].dt.time

In [111]:
df = df_met.merge(df_demand, on=['date', 'time'])

In [112]:
df.head()

,date,time,precipitation_total,atmospheric_pressure_station_level,atmospheric_pressure_sea_level,atmospheric_pressure_max_prev_hour,atmospheric_pressure_min_prev_hour,global_radiation,station_CPU_temp,air_temp_dry_bulb,...,station_battery_voltage,max_relative_humidity_prev_hour,min_relative_humidity_prev_hour,relative_humidity,wind_direction,wind_max_gust,wind_speed,week_day,energy_demand,is_weekend
0,2023-01-01,00:00:00,0.0,1011.1,1014.2,1011.3,1010.7,-1.7,26.0,25.9,...,12.5,100.000000,67.000000,67.000000,352.0,7.3,2.0,6.0,32183.885,True
1,2023-01-01,01:00:00,0.0,1011.3,1014.5,1011.5,1011.1,-3.3,26.0,23.1,...,12.5,80.000000,65.000000,80.000000,252.0,5.5,1.0,6.0,32196.682,True
2,2023-01-01,02:00:00,0.0,1011.2,1014.4,1011.4,1011.2,-3.3,25.0,23.0,...,12.5,78.571429,73.265306,75.877551,231.0,3.8,1.3,6.0,31539.570,True
3,2023-01-01,03:00:00,0.0,1010.6,1013.8,1011.2,1010.6,-3.5,25.0,22.7,...,12.5,78.591837,73.918367,76.354167,274.0,2.4,1.4,6.0,30699.367,True
4,2023-01-01,04:00:00,0.0,1009.7,1012.9,1010.6,1009.7,-3.5,24.0,23.2,...,12.5,78.282609,74.369565,77.425532,6.0,2.9,0.6,6.0,30126.195,True
